In [1]:
import pandas as pd
import numpy as np
import torch

import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from cleaning.pipeline import strategic_fill, shift_forecast_columns

In [2]:

project_root = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
data_path = os.path.join(project_root, 'data', 'raw_data', 'energy_dataset.csv')
print(f"Trying to load from: {data_path}")
raw_data = pd.read_csv(data_path)

raw_data['time'] = pd.to_datetime(raw_data['time'], utc=True)
raw_data.set_index('time', inplace=True)

Trying to load from: /Users/aryanahri/epf_datasets/data/raw_data/energy_dataset.csv


In [3]:
raw_data.head()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
time,,,,,,,,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,NaN,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
2015-01-01 00:00:00+00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,NaN,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2015-01-01 01:00:00+00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,NaN,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
2015-01-01 02:00:00+00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,NaN,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
2015-01-01 03:00:00+00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,NaN,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [4]:
start_time = raw_data.index.min()
end_time = raw_data.index.max()
complete_range = pd.date_range(start=start_time, end=end_time, freq='h')

# Find missing hours and then insert these times to make the time consecutive
missing_hours = complete_range.difference(raw_data.index)
print(missing_hours)
missing_df = pd.DataFrame(index=missing_hours)
raw_data = pd.concat([raw_data, missing_df])
raw_data = raw_data.sort_index()

filled_df = strategic_fill(raw_data)

DatetimeIndex([], dtype='datetime64[ns, UTC]', freq='h')


In [5]:
filled_df = filled_df.iloc[1:]

In [6]:
filled_df.head()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,NaN,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2015-01-01 01:00:00+00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,NaN,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
2015-01-01 02:00:00+00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,NaN,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
2015-01-01 03:00:00+00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,NaN,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04
2015-01-01 04:00:00+00:00,410.0,178.0,0.0,4038.0,3590.0,156.0,0.0,0.0,0.0,NaN,...,188.0,0.0,4618.0,4.0,NaN,4617.0,21441.0,19905.0,35.72,53.63


In [7]:
columns_to_drop = ['generation hydro pumped storage aggregated', 'forecast wind onshore day ahead', 'forecast wind offshore eday ahead']
filled_df = filled_df.drop(columns=columns_to_drop, errors='ignore')

In [8]:
filled_df.head()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage consumption,...,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,total load forecast,total load actual,price day ahead,price actual
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,920.0,...,71.0,50.0,195.0,0.0,5890.0,16.0,24934.0,24382.0,48.10,64.92
2015-01-01 01:00:00+00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,1164.0,...,73.0,50.0,196.0,0.0,5461.0,8.0,23515.0,22734.0,47.33,64.48
2015-01-01 02:00:00+00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,1503.0,...,75.0,50.0,191.0,0.0,5238.0,2.0,22642.0,21286.0,42.27,59.32
2015-01-01 03:00:00+00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,1826.0,...,74.0,42.0,189.0,0.0,4935.0,9.0,21785.0,20264.0,38.41,56.04
2015-01-01 04:00:00+00:00,410.0,178.0,0.0,4038.0,3590.0,156.0,0.0,0.0,0.0,2109.0,...,74.0,34.0,188.0,0.0,4618.0,4.0,21441.0,19905.0,35.72,53.63


In [9]:
null_indices = filled_df.index[filled_df.isnull().any(axis=1)]
print("Indices with null values:")
print(null_indices)

print("\nColumns at each index with null values:")
for idx in null_indices:
    print(f"\nIndex: {idx}")
    print(filled_df.loc[idx])

Indices with null values:
DatetimeIndex([], dtype='datetime64[ns, UTC]', name='time', freq=None)

Columns at each index with null values:


In [10]:
# Hour of the Day (0 to 23)
filled_df['HoD'] = filled_df.index.hour
# Day of the Week (1=Monday to 7=Sunday)
filled_df['DoW'] = filled_df.index.dayofweek + 1
filled_df['MoY'] = filled_df.index.month

In [11]:
filled_df[['HoD', 'DoW', 'MoY']].head()

,HoD,DoW,MoY
time,,,
2015-01-01 00:00:00+00:00,0,4,1
2015-01-01 01:00:00+00:00,1,4,1
2015-01-01 02:00:00+00:00,2,4,1
2015-01-01 03:00:00+00:00,3,4,1
2015-01-01 04:00:00+00:00,4,4,1


In [12]:
frc_cols = [col for col in filled_df.columns if "forecast" in col]
print(frc_cols)

['forecast solar day ahead', 'total load forecast']


In [13]:
shifted_df = shift_forecast_columns(df=filled_df, forecast_cols=frc_cols, shift_hours=-24)

In [14]:
frc_cols = frc_cols + ["total load actual"]
print(frc_cols)
shifted_df[frc_cols].head()

['forecast solar day ahead', 'total load forecast', 'total load actual']


,forecast solar day ahead,total load forecast,total load actual
time,,,
2015-01-01 00:00:00+00:00,35.0,25397.0,24382.0
2015-01-01 01:00:00+00:00,43.0,23640.0,22734.0
2015-01-01 02:00:00+00:00,32.0,22638.0,21286.0
2015-01-01 03:00:00+00:00,31.0,22238.0,20264.0
2015-01-01 04:00:00+00:00,26.0,22299.0,19905.0


In [15]:
# shifted_df.to_csv(f'{project_root}/data/cleaned/cleaned_spain_data.csv')

In [16]:
print(len(list(shifted_df.columns)))

28


In [17]:
print(len(shifted_df))

35039


In [18]:
row_index = 0
date_at_row = shifted_df.index[row_index]
print(f"Date at row {row_index}: {date_at_row}")

Date at row 0: 2015-01-01 00:00:00+00:00


In [19]:
row_index = 21024
date_at_row = shifted_df.index[row_index]
print(f"Date at row {row_index}: {date_at_row}")

Date at row 21024: 2017-05-26 00:00:00+00:00


In [20]:
row_index = 21025
date_at_row = shifted_df.index[row_index]
print(f"Date at row {row_index}: {date_at_row}")

Date at row 21025: 2017-05-26 01:00:00+00:00


In [21]:
row_index = 21024 + 7007
date_at_row = shifted_df.index[row_index]
print(f"Date at row {row_index}: {date_at_row}")

Date at row 28031: 2018-03-13 23:00:00+00:00


In [22]:
row_index = 21024 + 7007 + 1
date_at_row = shifted_df.index[row_index]
print(f"Date at row {row_index}: {date_at_row}")

Date at row 28032: 2018-03-14 00:00:00+00:00


In [23]:
row_index = 21024 + 7007 + 7007
date_at_row = shifted_df.index[row_index]
print(f"Date at row {row_index}: {date_at_row}")

Date at row 35038: 2018-12-30 22:00:00+00:00


In [24]:
list(shifted_df.columns)

['generation biomass',
 'generation fossil brown coal/lignite',
 'generation fossil coal-derived gas',
 'generation fossil gas',
 'generation fossil hard coal',
 'generation fossil oil',
 'generation fossil oil shale',
 'generation fossil peat',
 'generation geothermal',
 'generation hydro pumped storage consumption',
 'generation hydro run-of-river and poundage',
 'generation hydro water reservoir',
 'generation marine',
 'generation nuclear',
 'generation other',
 'generation other renewable',
 'generation solar',
 'generation waste',
 'generation wind offshore',
 'generation wind onshore',
 'forecast solar day ahead',
 'total load forecast',
 'total load actual',
 'price day ahead',
 'price actual',
 'HoD',
 'DoW',
 'MoY']

In [25]:
# Check for NaN values in the dataframe
nan_count = shifted_df.isna().sum()
print("Count of NaN values in each column:")
print(nan_count)

# Check if there are any NaN values in the entire dataframe
has_nan = shifted_df.isna().any().any()
print(f"\nDataframe contains NaN values: {has_nan}")

# If there are NaN values, show rows with NaN
if has_nan:
    print("\nSample rows containing NaN values:")
    nan_rows = shifted_df[shifted_df.isna().any(axis=1)]
    print(nan_rows.head())
    print(f"Total rows with NaN values: {len(nan_rows)}")


Count of NaN values in each column:
generation biomass                             0
generation fossil brown coal/lignite           0
generation fossil coal-derived gas             0
generation fossil gas                          0
generation fossil hard coal                    0
generation fossil oil                          0
generation fossil oil shale                    0
generation fossil peat                         0
generation geothermal                          0
generation hydro pumped storage consumption    0
generation hydro run-of-river and poundage     0
generation hydro water reservoir               0
generation marine                              0
generation nuclear                             0
generation other                               0
generation other renewable                     0
generation solar                               0
generation waste                               0
generation wind offshore                       0
generation wind onshore          

In [26]:
# Check for non-numeric features in the dataframe
non_numeric_columns = []

for column in shifted_df.columns:
    # Check if the column contains non-numeric data
    if not pd.api.types.is_numeric_dtype(shifted_df[column]):
        non_numeric_columns.append(column)

if non_numeric_columns:
    print("The following columns contain non-numeric data:")
    for col in non_numeric_columns:
        print(f"- {col}: {shifted_df[col].dtype}")
    
    # Display sample of non-numeric columns
    print("\nSample values from non-numeric columns:")
    for col in non_numeric_columns:
        print(f"\n{col}:")
        print(shifted_df[col].head())
else:
    print("All columns in the dataframe contain numeric data.")


All columns in the dataframe contain numeric data.
